In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

21/08/28 17:06:47 WARN Utils: Your hostname, DESKTOP-C98VC8U resolves to a loopback address: 127.0.1.1; using 172.23.181.208 instead (on interface eth0)
21/08/28 17:06:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/08/28 17:06:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
customers = spark.read.parquet("./output/customers.parquet")
customers.show(3)

+----------------+---------+--------+--------------------+--------+--------------+
|CustomerIdentity|FirstName|LASTNAME|               phone|postcode|Customer_Title|
+----------------+---------+--------+--------------------+--------+--------------+
|               1|    Terri| Leonard|          3124065463|   65369|          Mrs.|
|               2|  Maurice|    West|001-468-075-5301x308|   27061|           Dr.|
|               3|   Taylor|    Hall|    001-004-938-2651|   88302|           Mr.|
+----------------+---------+--------+--------------------+--------+--------------+
only showing top 3 rows



In [3]:
tickets = spark.read.parquet("./output/tickets.parquet")
tickets.show(3)

+---------+--------+-----------+--------+---------+----------+------------------+------------+----------+
|ticket_id|order_id|customer_id|quantity|Net_sales|event_code|        event_name|event_season|      Date|
+---------+--------+-----------+--------+---------+----------+------------------+------------+----------+
|        1|     182|        160|       1|    58.55|   CHL-ARS|Chelsea vs Arsenal|   2020/2021|2018-05-05|
|        2|     376|         46|       1|    44.25|   CHL-ARS|Chelsea vs Arsenal|   2020/2021|2018-05-05|
|        3|     282|        116|       5|     47.0|   CHL-ARS|Chelsea vs Arsenal|   2020/2021|2018-05-05|
+---------+--------+-----------+--------+---------+----------+------------------+------------+----------+
only showing top 3 rows



In [4]:
tickets.printSchema()

root
 |-- ticket_id: integer (nullable = true)
 |-- order_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- Net_sales: double (nullable = true)
 |-- event_code: string (nullable = true)
 |-- event_name: string (nullable = true)
 |-- event_season: string (nullable = true)
 |-- Date: date (nullable = true)



In [5]:
#broadcast joining because one table is very small, reduces shuffling
import pyspark.sql.functions as F
join_condition = customers.CustomerIdentity == tickets.customer_id
combined_df =  tickets.join(F.broadcast(customers), join_condition).drop("CustomerIdentity")
combined_df.cache()

DataFrame[ticket_id: int, order_id: int, customer_id: int, quantity: int, Net_sales: double, event_code: string, event_name: string, event_season: string, Date: date, FirstName: string, LASTNAME: string, phone: string, postcode: string, Customer_Title: string]

In [9]:
print(len(combined_df.columns), combined_df.count())

14 1000


In [10]:
combined_df.select("Date").where(F.col("Date").isNull()).count()

0

In [52]:
combined_df.show(3, truncate=False)

+---------+--------+-----------+--------+---------+----------+------------------+------------+----------+---------+--------+----------------------+--------+--------------+
|ticket_id|order_id|customer_id|quantity|Net_sales|event_code|event_name        |event_season|Date      |FirstName|LASTNAME|phone                 |postcode|Customer_Title|
+---------+--------+-----------+--------+---------+----------+------------------+------------+----------+---------+--------+----------------------+--------+--------------+
|1        |182     |160        |1       |58.55    |CHL-ARS   |Chelsea vs Arsenal|2020/2021   |2018-05-05|Tony     |Baird   |001-097-596-0629x46835|94414   |Ms.           |
|2        |376     |46         |1       |44.25    |CHL-ARS   |Chelsea vs Arsenal|2020/2021   |2018-05-05|Justin   |Higgins |1889149702            |67579   |Mrs.          |
|3        |282     |116        |5       |47.0     |CHL-ARS   |Chelsea vs Arsenal|2020/2021   |2018-05-05|Jennifer |Dawson  |479.687.6222x409

In [12]:
combined_df.select("customer_id").distinct().count()

198

In [13]:
combined_df.createOrReplaceTempView("DF")

In [14]:
spark.sql("""
          SELECT order_id, Date 
          from DF 
          where Date == "2018-01-30"
          """).show(2)

+--------+----------+
|order_id|      Date|
+--------+----------+
|      20|2018-01-30|
|      14|2018-01-30|
+--------+----------+
only showing top 2 rows



In [70]:
spark.sql("""
          drop database trial
          """)

DataFrame[]

In [60]:
transcation = combined_df.select("order_id", "ticket_id", "customer_id", "event_code", "Date",  "quantity", "Net_sales").distinct().orderBy("Date")
transcation.show(3)

+--------+---------+----------+----------+-----------+--------+---------+
|order_id|ticket_id|event_code|      Date|customer_id|quantity|Net_sales|
+--------+---------+----------+----------+-----------+--------+---------+
|     138|      108|   BTH-SRC|2018-01-30|        145|       5|    40.14|
|     289|      260|   F1-SILV|2018-01-30|         42|       2|    13.22|
|     370|      343|   BTH-SRC|2018-01-30|         55|       3|    10.33|
+--------+---------+----------+----------+-----------+--------+---------+
only showing top 3 rows



In [58]:
event = combined_df.select("event_code", "event_name", "event_season").distinct()
event.show()

+----------+------------------+------------+
|event_code|        event_name|event_season|
+----------+------------------+------------+
|   BTH-SRC|  Bath vs Saracens|   2020/2021|
|   CHL-ARS|Chelsea vs Arsenal|   2020/2021|
|   F1-SILV|       Silverstone|   2020/2021|
+----------+------------------+------------+



In [55]:
customer = combined_df.select("customer_id", "Customer_Title", "FirstName", "LASTNAME", "phone", "postcode").distinct().orderBy("customer_id")
customer.show(3)

+-----------+--------------+---------+--------+--------------------+--------+
|customer_id|Customer_Title|FirstName|LASTNAME|               phone|postcode|
+-----------+--------------+---------+--------+--------------------+--------+
|          1|          Mrs.|    Terri| Leonard|          3124065463|   65369|
|          2|           Dr.|  Maurice|    West|001-468-075-5301x308|   27061|
|          3|           Mr.|   Taylor|    Hall|    001-004-938-2651|   88302|
+-----------+--------------+---------+--------+--------------------+--------+
only showing top 3 rows



In [26]:
import pyspark.sql.functions as F
# 3. For each Customer, a list of Events
Edf = combined_df.groupBy("customer_id")\
                 .agg(F.collect_list(F.col("event_code"))\
                 .alias("Events_list"))
Edf.show(3, truncate=False)

+-----------+------------------------------------------------------------------------------------------+
|customer_id|Events_list                                                                               |
+-----------+------------------------------------------------------------------------------------------+
|148        |[BTH-SRC, CHL-ARS, CHL-ARS, F1-SILV, CHL-ARS]                                             |
|31         |[CHL-ARS, CHL-ARS, BTH-SRC, BTH-SRC, CHL-ARS, CHL-ARS, BTH-SRC, CHL-ARS, BTH-SRC, CHL-ARS]|
|85         |[CHL-ARS, F1-SILV, F1-SILV]                                                               |
+-----------+------------------------------------------------------------------------------------------+
only showing top 3 rows



In [28]:
# combined_df.groupBy("CustomerIdentity").agg(F.count("event_code").alias("Totals")).sort("Totals", ascending=True).show()
Tots = combined_df.groupBy("customer_id").agg(F.count("event_name").alias("Totals"))
Tots.show(3)

+-----------+------+
|customer_id|Totals|
+-----------+------+
|        148|     5|
|         31|    10|
|        137|     7|
+-----------+------+
only showing top 3 rows



In [38]:
MultiEvent = Tots.withColumn("MultiEvent", F.col("Totals") > 1)
MultiEvent.show(6)

+-----------+------+----------+
|customer_id|Totals|MultiEvent|
+-----------+------+----------+
|        148|     5|      true|
|         31|    10|      true|
|        137|     7|      true|
|         85|     3|      true|
|         65|     3|      true|
|         53|     1|     false|
+-----------+------+----------+
only showing top 6 rows



In [35]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *

multiEvent = lambda x: "true" if x>1 else " "

def multiEvent1(x):
    
   if x > 1:

        return   "True"
   else:
        return " "
    
multiEvent_udf = udf(multiEvent, StringType())
multiEvent_udf1 = udf(multiEvent1, StringType())

#List of **all** Customers with an additional column called "MultiEvent", set to `True` for those Customers with more than 1 Event
Tots.withColumn("MultiEvent", multiEvent_udf(F.col("Totals"))).show(5)

In [37]:
MultiEvent1 = Tots.withColumn("MultiEvent88", multiEvent_udf1(F.col("Totals")))
MultiEvent1.show(3)

+-----------+------+------------+
|customer_id|Totals|MultiEvent88|
+-----------+------+------------+
|        148|     5|        True|
|         31|    10|        True|
|        137|     7|        True|
+-----------+------+------------+
only showing top 3 rows



In [36]:
MultiEvent = Tots.withColumn("MultiEvent88", multiEvent_udf(F.col("Totals")))
MultiEvent.show(3)

+-----------+------+------------+
|customer_id|Totals|MultiEvent88|
+-----------+------+------------+
|        148|     5|        true|
|         31|    10|        true|
|        137|     7|        true|
+-----------+------+------------+
only showing top 3 rows



In [39]:
cols = ["customer_id", "event_name", "event_season", "Date", "rank"]
combined_df.groupBy("customer_id").agg(F.max("quantity")).show()

+-----------+-------------+
|customer_id|max(quantity)|
+-----------+-------------+
|        148|            4|
|         31|            5|
|        137|            5|
|         85|            5|
|         65|            4|
|         53|            2|
|        133|            5|
|         78|            4|
|        108|            5|
|        155|            5|
|         34|            4|
|        193|            5|
|        115|            5|
|        101|            5|
|        126|            5|
|         81|            3|
|        183|            5|
|         28|            3|
|         76|            5|
|         26|            5|
+-----------+-------------+
only showing top 20 rows



In [44]:
# 5. Largest Order by Quantity for each Customer
from pyspark.sql.window import Window
cols = ["customer_id", "quantity"]
windowSpec1 = Window.partitionBy(["customer_id"])\
                    .orderBy(F.desc("quantity"))

ranked = combined_df.withColumn("rank", F.rank().over(windowSpec1)).filter("rank == 1").select(cols)
ranked.show(5)

+-----------+--------+
|customer_id|quantity|
+-----------+--------+
|        148|       4|
|         31|       5|
|         31|       5|
|         31|       5|
|         31|       5|
+-----------+--------+
only showing top 5 rows



In [50]:
ranked.drop_duplicates()\
        .orderBy(F.desc("quantity"))\
        .orderBy("customer_id").show()

+-----------+--------+
|customer_id|quantity|
+-----------+--------+
|          1|       3|
|          2|       4|
|          3|       5|
|          4|       5|
|          5|       4|
|          6|       5|
|          7|       5|
|          8|       5|
|          9|       5|
|         10|       5|
|         11|       4|
|         12|       3|
|         13|       4|
|         14|       3|
|         15|       2|
|         16|       5|
|         17|       4|
|         18|       2|
|         19|       5|
|         20|       5|
+-----------+--------+
only showing top 20 rows



In [79]:
from pyspark.sql import Window

WindowSpec = Window.partitionBy("customer_id").orderBy(F.desc("quantity"))

F = combined_df.withColumn("dddd", F.collect_list(F.col("quantity")).over(WindowSpec)).select("customer_id", "dddd")
F.show()

+-----------+--------------------+
|customer_id|                dddd|
+-----------+--------------------+
|        148|                 [4]|
|        148|        [4, 3, 3, 3]|
|        148|        [4, 3, 3, 3]|
|        148|        [4, 3, 3, 3]|
|        148|     [4, 3, 3, 3, 2]|
|         31|        [5, 5, 5, 5]|
|         31|        [5, 5, 5, 5]|
|         31|        [5, 5, 5, 5]|
|         31|        [5, 5, 5, 5]|
|         31|  [5, 5, 5, 5, 4, 4]|
|         31|  [5, 5, 5, 5, 4, 4]|
|         31|[5, 5, 5, 5, 4, 4...|
|         31|[5, 5, 5, 5, 4, 4...|
|         31|[5, 5, 5, 5, 4, 4...|
|         31|[5, 5, 5, 5, 4, 4...|
|         85|                 [5]|
|         85|              [5, 4]|
|         85|           [5, 4, 1]|
|        137|              [5, 5]|
|        137|              [5, 5]|
+-----------+--------------------+
only showing top 20 rows



In [ ]:
F.collect()[2]["dddd"]

In [59]:
from pyspark.sql import Window

WindowSpec = Window.partitionBy("customer_id")

df22 = combined_df.withColumn("EventsList", 
                              F.collect_list(F.col("event_code"))
                              .over(WindowSpec)
                              )\
.select("customer_id", "EventsList")
df22.show(5, truncate=False)



+-----------+---------------------------------------------+
|customer_id|EventsList                                   |
+-----------+---------------------------------------------+
|148        |[CHL-ARS, BTH-SRC, CHL-ARS, CHL-ARS, F1-SILV]|
|148        |[CHL-ARS, BTH-SRC, CHL-ARS, CHL-ARS, F1-SILV]|
|148        |[CHL-ARS, BTH-SRC, CHL-ARS, CHL-ARS, F1-SILV]|
|148        |[CHL-ARS, BTH-SRC, CHL-ARS, CHL-ARS, F1-SILV]|
|148        |[CHL-ARS, BTH-SRC, CHL-ARS, CHL-ARS, F1-SILV]|
+-----------+---------------------------------------------+
only showing top 5 rows



In [16]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="event_code", outputCol="IndexEvents")
indexed = indexer.fit(combined_df).transform(combined_df)
indexed.show(10)

+---------+--------+--------+---------+----------+------------------+------------+----------+----------------+---------+--------+--------------------+--------+--------------+-----------+
|ticket_id|order_id|quantity|Net_sales|event_code|        event_name|event_season|      Date|CustomerIdentity|FirstName|LASTNAME|               phone|postcode|Customer_Title|IndexEvents|
+---------+--------+--------+---------+----------+------------------+------------+----------+----------------+---------+--------+--------------------+--------+--------------+-----------+
|        1|     182|       1|    58.55|   CHL-ARS|Chelsea vs Arsenal|   2020/2021|2018-05-05|             160|     Tony|   Baird|001-097-596-0629x...|   94414|           Ms.|        0.0|
|        2|     376|       1|    44.25|   CHL-ARS|Chelsea vs Arsenal|   2020/2021|2018-05-05|              46|   Justin| Higgins|          1889149702|   67579|          Mrs.|        0.0|
|        3|     282|       5|     47.0|   CHL-ARS|Chelsea vs Arse

In [18]:
combined_df.groupBy("Customer_Title").agg(F.sum("quantity").alias("Totals")).sort("Totals", ascending=False).show()

+--------------+------+
|Customer_Title|Totals|
+--------------+------+
|           Dr.|  1023|
|           Mr.|   996|
|          Mrs.|   691|
|          Miss|   131|
|           Ms.|   128|
+--------------+------+



In [27]:
add = lambda x,y : x+y
add(2,3)

5

In [14]:
def display(name, age):
    print('My name and age is {}, {}'.format(name, age))


display("Joe", 25)

My name and ag is oe, 25


In [10]:
def outer(x, y):
    s = x+y
    
    def inner():
        return 2*s
        
    return inner

my = outer(2,3)
my()


10

In [22]:
#List of **all** Customers with an additional column called "MultiEvent", set to `True` for those Customers with more than 1 Event
combined_df.groupBy("customer_id").agg(F.count("event_code").alias("Totals")).sort("Totals", ascending=True).show()

+-----------+------+
|customer_id|Totals|
+-----------+------+
|         53|     1|
|         86|     1|
|        121|     1|
|        126|     1|
|        175|     1|
|         28|     1|
|        149|     1|
|        145|     2|
|         15|     2|
|        156|     2|
|          5|     2|
|         61|     2|
|         24|     2|
|         62|     2|
|          1|     2|
|        117|     2|
|        171|     2|
|         13|     2|
|         81|     2|
|         91|     2|
+-----------+------+
only showing top 20 rows

